In [1]:
import sys
if not '..' in sys.path:
    sys.path += ['..']
    
import torch
from torch import nn
from data.dataset import get_data
import matplotlib.pyplot as plt
from model.model import Transfollower, lstm_model
from config import Settings
settings = Settings()

In [8]:
MODEL = 'nn' # transfollower
DATASET = 'NGSIM' # SH
exp_name = f'{DATASET}_{MODEL}'
save = f'{exp_name}_model.pt'

with open(f'../checkpoints/{save}', 'rb') as f:
    model = torch.load(f)

In [9]:
model.eval()

_, _, test_loader = get_data(data_name = 'NGSIM')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.MSELoss()
model = model.to(device)

with torch.no_grad():
    item = next(iter(test_loader))
    
    enc_inp = item['his'].float().to(device)

    batch_y = item['svSpd'].float()
    y_label = batch_y[:,-settings.PRED_LEN:,:].to(device)
    batch_y_mark = item['lvSpd'].float().to(device)
    # decoder input
    dec_inp = torch.zeros([batch_y.shape[0], settings.PRED_LEN, batch_y.shape[-1]]).float() + \
            batch_y[:,:settings.LABEL_LEN,:].mean(axis = 1, keepdim=True)
    dec_inp = torch.cat([batch_y[:,:settings.LABEL_LEN,:], dec_inp], dim=1).float().to(device)
    dec_inp = torch.cat([dec_inp, batch_y_mark], axis = -1) # adding lv speed

    # encoder - decoder
    out = model(enc_inp, dec_inp)

    lvSpd, spacing = item['lvSpd'][:, settings.LABEL_LEN:,:].float().to(device), item['spacing'].float().to(device)
    relSpd_ = (lvSpd - out).squeeze()
    spacing_ = torch.cumsum(settings.T*(relSpd_[:,:-1] + relSpd_[:,1:])/2, dim = -1) + item['s0'].float().to(device)
    loss = criterion(out, y_label) + criterion(spacing_, spacing)
    svSpd_ = out
    
    his, lvSpd, svSpd, spacing = item['his'].float().to(device), item['lvSpd'].float().to(device), \
    item['svSpd'].float().to(device), item['spacing'].float().to(device)
    print(loss.item())
    
for i in range(100):
    plt.figure();
    plt.plot(batch_y_mark[i,settings.LABEL_LEN:,:].detach().cpu(), label = 'lv');
    plt.plot(svSpd_[i].detach().cpu(),'--', label = 'predict');
    plt.plot(y_label[i].detach().cpu(), label = 'observed');
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('SV Speed (m/s)')
    
    plt.figure();
    plt.plot(spacing_[i].detach().cpu(), '--',label = 'predict');
    plt.plot(spacing[i].detach().cpu(), label = 'observed');
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Spacing (m)')

NGSIM, total data samples for train, val, and test
938 201 202
Total batches for train, val, and test with batch size 256
4 1 1


TypeError: forward() takes 2 positional arguments but 3 were given